# ¿Cuál es la mejor tarifa?

Trabajas como analista para el operador de telecomunicaciones Megaline. La empresa ofrece a sus clientes dos tarifas de prepago, Surf y Ultimate. El departamento comercial quiere saber cuál de las tarifas genera más ingresos para poder ajustar el presupuesto de publicidad.

Vas a realizar un análisis preliminar de las tarifas basado en una selección de clientes relativamente pequeña. Tendrás los datos de 500 clientes de Megaline: quiénes son los clientes, de dónde son, qué tarifa usan, así como la cantidad de llamadas que hicieron y los mensajes de texto que enviaron en 2018. Tu trabajo es analizar el comportamiento de los clientes y determinar qué tarifa de prepago genera más ingresos.

# Contenido <a id='back'></a>

* [Introducción](#intro)
* [Paso 1. Descripción de los datos](#data_review)
    * [1 Diccionario de datos](#dictionary)
    * [2 Plan de solucion](#plan1)
    * [3 Conclusiones](#data_review_conclusions)
* [Paso 2. Preprocesamiento de datos](#data_preprocessing)
    * [4 Plan de solucion](#plan2)
    * [5 Valores duplicados](#duplicates)
    * [6 Valores ausentes](#missing)
    * [7 Conclusiones](#data_preprocessing_conclusions)
* [Paso 3. Análisis de los datos](#analisys)
    * [A. Fácil](#easy)
    * [B. Intermedio](#inter)
    * [C. Difícil](#hard)
* [Conclusiones](#end)

# Introducción <a id='intro'></a>

La empresa operadora de Telecomunicaciones MEGALINE ofrece a sus clientes dos tarifas de prepago, Surf y Ultimate. Necesitamos saber cuál de los dos planes genera más ingresos para ajustar el presupuesto de publicidad, de acuerdo a los requerimientos del departamento comercial de la compañía.

Vamos a realizar un análisis preliminar de las tarifas basado en una selección de clientes relativamente pequeña. Tendrás los datos de 500 clientes de Megaline: quiénes son los clientes, de dónde son, qué tarifa usan, así como la cantidad de llamadas que hicieron y los mensajes de texto que enviaron en 2018. Tu trabajo es analizar el comportamiento de los clientes y determinar qué tarifa de prepago genera más ingresos. Más adelante, encontrarás en las instrucciones del proyecto cuáles son exactamente los aspectos del comportamiento de los clientes que debes analizar. Determinar qué plan, en promedio, aporta más ingresos es una cuestión que se abordará mediante pruebas estadísticas. Más adelante encontrarás más información al respecto en la sección de instrucciones del proyecto.

Descripción de las tarifas:

Nota: Megaline redondea los segundos a minutos y los megabytes a gigabytes. Para las llamadas, cada llamada individual se redondea: incluso si la llamada duró solo un segundo, se contará como un minuto. Para el tráfico web, las sesiones web individuales no se redondean. En vez de esto, el total del mes se redondea hacia arriba. Si alguien usa 1025 megabytes este mes, se le cobrarán 2 gigabytes.

A continuación, puedes ver una descripción de las tarifas:

Surf

1.	Pago mensual: $20 usd.
2.	500 minutos al mes, 50 SMS y 15 GB de datos.
3.	Si se exceden los límites del paquete:

    o	1 minuto: 3 centavos.

    o	1 SMS: 3 centavos.

    o	1 GB de datos: $10 usd.

Ultimate

1.	Pago mensual: $70 usd.
2.	3000 minutos al mes, 1000 SMS y 30 GB de datos.
3.	Si se exceden los límites del paquete:

    o	1 minuto: 1 centavo.

    o	1 SMS: 1 centavo.
    
    o	1 GB de datos: $7 usd.

[Volver a Contenidos](#back)

[Te proporcionamos algunos comentarios para orientarte mientras completas este proyecto. Pero debes asegurarte de eliminar todos los comentarios entre corchetes antes de entregar tu proyecto.]

[Antes de sumergirte en el análisis de datos, explica por tu propia cuenta el propósito del proyecto y las acciones que planeas realizar.]

[Ten en cuenta que estudiar, modificar y analizar datos es un proceso iterativo. Es normal volver a los pasos anteriores y corregirlos/ampliarlos para permitir nuevos pasos.]

# Paso 1. Descripción de los datos <a id='data_review'></a>

Para este trabajo se tienen los siguientes 5 archivos de datos (`/datasets/megaline_users.csv `, `/datasets/megaline_plans.csv `, `/datasets/megaline_calls.csv `, `/datasets/megaline_internet.csv ` y `/datasets/megaline_messages.csv `) los cuales se leeran con la librería Pandas de Python, utilizando el método de lectura de archivos separados por comas `pd.read_csv()` usando el parámetro sep=';' para leer los datos correctamente. 
De primer instancia, se verificará la información para cada uno de los DataFrames.

## Diccionario de datos <a id='dictionary'></a>

Contamos con cinco Datasets en el conjunto de datos, usaremos todas para hacer el preprocesamiento de los datos y el análisis exploratorio de los datos.
A continuación se muestra el diccionario de datos que enumeroa las columnas de cada uno de los Datasets y se describen los datos que contienen.

- `megaline_users.csv`: cada fila corresponde a un usuario en la Compañia Megaline (datos sobre los usuarios).
    - `'user_id'`: identificador único del usuario.
    - `'first_name'`: nombre del usuario.
    - `'last_name'`: apellido del usuario.
    - `'age'`: edad del usuario (en años).
    - `'reg_date'`: fecha de suscripción (dd, mm, aa).
    - `'churn_date'`: la fecha en la que el usuario dejó de usar el servicio (si el valor es ausente, la tarifa se estaba usando cuando fue extraída esta base de datos).
    - `'city'`: ciudad de residencia del usuario.
    - `'plan'`: nombre de la tarifa ('surf' o 'ultimate')

- `megaline_plans.csv `: cada fila corresponde a... (datos sobre las tarifas).
    - `'plan_name'`: nombre de la tarifa.
    - `'usd_monthly_fee'`: pago mensual en dólares estadounidenses.
    - `'minutes_included'`: minutos incluidos al mes.
    - `'messages_included'`: SMS incluidos al mes.
    - `'mb_per_month_included'`: datos incluidos al mes (en megabytes).
    - `'usd_per_minute'`: precio por minuto tras exceder los límites del paquete (por ejemplo, si el paquete incluye 100 minutos, el operador cobrará el minuto 101).
    - `'usd_per_message'`: precio por SMS tras exceder los límites del paquete.
    - `'usd_per_gb'`: precio por gigabyte de los datos extra tras exceder los límites del paquete (1 GB = 1024 megabytes).

- `megaline_calls.csv`: cada fila corresponde a una llamada única que realizaron los clientes (datos sobre las llamadas).
    - `'id'`: identificador único de la llamada.
    - `'call_date'`: fecha de la llamada.
    - `'duration'`: duración de la llamada (en minutos).
    - `'user_id'`: el identificador del usuario que realiza la llamada.

- `megaline_internet.csv`: cada fila corresponde a una consulta en internet que realizaron los clientes (datos sobre las sesiones web).
    - `'id'`: identificador único de la sesión.
    - `'mb_used'`: el volumen de datos gastados durante la sesión (en megabytes).
    - `'session_date'`: fecha de la sesión web.
    - `'user_id'`: identificador del usuario.

- `megaline_messages.csv`: cada fila corresponde a un mensaje tipo sms que realizaron los clientes (datos sobre los SMS).
    - `'id'`: identificador único del SMS.
    - `'message_date'`: fecha del SMS.
    - `'user_id'`: el identificador del usuario que manda el SMS.

[Volver a Contenidos](#back)

## Plan de solución <a id='plan1'></a>

Descripción de los datos.

Para iniciar con la revisión de todos los Datasets, vamos a importa las librerias de Pandas, Numpy y Stats de Scipy que nos serviran para crear un Dataframe por cada uno de los archivos csv.

De esta manera será posible revisar la información contenida en cada archivo, realizando el Preprocesamiento de los datos; como corrección de los nombres de las columnas para cumplir con el formato SnakeCase; rellenar valores ausentes, y eliminar datos duplicados de ser necesario.

Finalmente, realizaremos el Análisis de los datos, creando Hipótesis que nos ayuden a tomar buenas desiciones con la información qye tenemos.

## Inicialización

In [1]:
# Cargar todas las librerías
import pandas as pd
import numpy as np
from scipy import stats as st 

## Cargar datos

In [2]:
# Carga los archivos de datos en diferentes DataFrames
df_users = pd.read_csv(r'C:\Users\10147115\Python Project Folder\TripleTen_DS\Sprint 5_Análisis estadístico de datos\Proyecto SP5\Datasets\megaline_users.csv', sep=',')
df_plans = pd.read_csv(r'C:\Users\10147115\Python Project Folder\TripleTen_DS\Sprint 5_Análisis estadístico de datos\Proyecto SP5\Datasets\megaline_plans.csv', sep=',')
df_calls = pd.read_csv(r'C:\Users\10147115\Python Project Folder\TripleTen_DS\Sprint 5_Análisis estadístico de datos\Proyecto SP5\Datasets\megaline_calls.csv', sep=',')
df_data = pd.read_csv(r'C:\Users\10147115\Python Project Folder\TripleTen_DS\Sprint 5_Análisis estadístico de datos\Proyecto SP5\Datasets\megaline_internet.csv', sep=',')
df_sms = pd.read_csv(r'C:\Users\10147115\Python Project Folder\TripleTen_DS\Sprint 5_Análisis estadístico de datos\Proyecto SP5\Datasets\megaline_messages.csv', sep=',')

## Preparar los datos

[Los datos para este proyecto se dividen en varias tablas. Explora cada una para tener una comprensión inicial de los datos. Si es necesario, haz las correcciones requeridas en cada tabla.]

## Tarifas

In [3]:
# Imprime la información general/resumida sobre el DataFrame de las tarifas
df_plans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   messages_included      2 non-null      int64  
 1   mb_per_month_included  2 non-null      int64  
 2   minutes_included       2 non-null      int64  
 3   usd_monthly_pay        2 non-null      int64  
 4   usd_per_gb             2 non-null      int64  
 5   usd_per_message        2 non-null      float64
 6   usd_per_minute         2 non-null      float64
 7   plan_name              2 non-null      object 
dtypes: float64(2), int64(5), object(1)
memory usage: 260.0+ bytes


In [4]:
# Imprime una muestra de los datos para las tarifas
df_plans.head()


,messages_included,mb_per_month_included,minutes_included,usd_monthly_pay,usd_per_gb,usd_per_message,usd_per_minute,plan_name
0,50,15360,500,20,10,0.03,0.03,surf
1,1000,30720,3000,70,7,0.01,0.01,ultimate


## Descripción

El dataframe de los planes contiene únicamente 2 filas y 8 columnas, con información sobre las tarifas que se ofrecen.

La mayoría de los datos de las columnas son de tipo números enteros o integer (int64), hay dos columnas con información monetaria (usd) que son de tipo números fraccionarios o float (float64), y finalmente una columna con información de tipo cualitativa o descriptiva (object) que son los nombres de los planes.

La información se ve bien pero podemos cambiar los tipos de datos en todas las columnas que contengan información monetaria en usd, para homologar todas las columnas con esta información.

No se tienen datos eliminados ni ausentes, los nombres de los planes están en minusculas, los nombres de las columnas cumplen el formato snake case, nombres en minúsculas, separación con guiones bajos, sin espacios al principio ni al final.

Por lo que no hay mucho que corregir en este primer dataframe revisado.

## Corregir datos

[Corrige los problemas obvios con los datos basándote en las observaciones iniciales.]

In [5]:
# Vamos a cambiar los Dtypes en todas las columnas con información monetaria (usd) de int64 a float64:
df_plans['usd_monthly_pay'] = df_plans['usd_monthly_pay'].astype('float64')
df_plans['usd_per_gb'] = df_plans['usd_per_gb'].astype('float64')
df_plans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   messages_included      2 non-null      int64  
 1   mb_per_month_included  2 non-null      int64  
 2   minutes_included       2 non-null      int64  
 3   usd_monthly_pay        2 non-null      float64
 4   usd_per_gb             2 non-null      float64
 5   usd_per_message        2 non-null      float64
 6   usd_per_minute         2 non-null      float64
 7   plan_name              2 non-null      object 
dtypes: float64(4), int64(3), object(1)
memory usage: 260.0+ bytes


## Enriquecer los datos

[Agrega factores adicionales a los datos si crees que pudieran ser útiles.]

In [6]:
# Agregamos una columna que contenga el los Mb (Megabytes) incluídos en Gb (Gigabytes)
gb_included = (df_plans['mb_per_month_included'] / 1024).astype('int64')
df_plans.insert(2, 'gb_included', gb_included)
df_plans

,messages_included,mb_per_month_included,gb_included,minutes_included,usd_monthly_pay,usd_per_gb,usd_per_message,usd_per_minute,plan_name
0,50,15360,15,500,20.0,10.0,0.03,0.03,surf
1,1000,30720,30,3000,70.0,7.0,0.01,0.01,ultimate


## Usuarios/as

In [7]:
# Imprime la información general/resumida sobre el DataFrame de usuarios
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     500 non-null    int64 
 1   first_name  500 non-null    object
 2   last_name   500 non-null    object
 3   age         500 non-null    int64 
 4   city        500 non-null    object
 5   reg_date    500 non-null    object
 6   plan        500 non-null    object
 7   churn_date  34 non-null     object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB


In [8]:
# Imprime una muestra de datos para usuarios
df_users.sample(5)

,user_id,first_name,last_name,age,city,reg_date,plan,churn_date
41,1041,Drucilla,Lynn,58,"Las Vegas-Henderson-Paradise, NV MSA",2018-03-17,ultimate,NaN
146,1146,Loma,Mcneil,42,"New York-Newark-Jersey City, NY-NJ-PA MSA",2018-10-13,surf,NaN
295,1295,Hung,Flowers,68,"Kansas City, MO-KS MSA",2018-08-11,surf,NaN
380,1380,Lajuana,Kelley,35,"San Jose-Sunnyvale-Santa Clara, CA MSA",2018-09-04,ultimate,NaN
47,1047,Yuk,French,22,"Indianapolis-Carmel-Anderson, IN MSA",2018-06-16,ultimate,NaN


En el dataframe de usuarios, podemos ver 8 columnas con diferentes datos, como 'user_id', 'first_name', 'last_name', 'age', 'city', 'reg_date', 'plan', 'churn_date'.

Al revisar los Dtypes podemos ver eu la columna con fecha de registro 'reg_date' y la columna 'churn_date', son de tipo object; lo cual no es lo más correcto y debe ser corregido a tipo de fecha y con formato de fecha.

También en la última columna llamada 'churn_date' solo tenemos 34 registros y los demás son valores ausentes, lo cual debe corregirse para evitar problemas en el análisis posterios, estos podrían sustituirse con un valor cero o desconocido (unknown).

La muestra a analizar es de 500 usuarios.

### Corregir los datos

Es necesario cambiar el formato de las columnas con fechas, para esto utilizaremos un método de Pandas llamado to_datetime.

Tanto la columna llamada 'reg_date' como la columna 'churn_date' se modificará su formato.

Rellenar los valores ausentes en la columna 'churn_date' con el nombre 'unknown'.

Renombraremos las columnas con nombres más descriptivos.

In [9]:
# cambiamos el formato a fecha en las columnas 'reg_date' y 'churn_date'
df_users['reg_date'] = pd.to_datetime(df_users['reg_date'], infer_datetime_format=True)
df_users['churn_date'] = pd.to_datetime(df_users['churn_date'], infer_datetime_format=True)

df_users['churn_date'] = df_users['churn_date'].fillna('Unknown')   # Rellenamos los valores ausentes
df_users.columns = ['user_id', 'first_name', 'last_name', 'user_age', 'user_city', 'reg_date', 'plan_name', 'churn_date']   # renombramos las columnas más descriptivas

df_users.info()     # verificamos la información general y los Dtypes de las columnas
df_users

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     500 non-null    int64         
 1   first_name  500 non-null    object        
 2   last_name   500 non-null    object        
 3   user_age    500 non-null    int64         
 4   user_city   500 non-null    object        
 5   reg_date    500 non-null    datetime64[ns]
 6   plan_name   500 non-null    object        
 7   churn_date  500 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 31.4+ KB


C:\Users\10147115\AppData\Local\Temp\ipykernel_13468\1625800408.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_users['reg_date'] = pd.to_datetime(df_users['reg_date'], infer_datetime_format=True)
C:\Users\10147115\AppData\Local\Temp\ipykernel_13468\1625800408.py:3: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_users['churn_date'] = pd.to_datetime(df_users['churn_date'], infer_datetime_format=True)


,user_id,first_name,last_name,user_age,user_city,reg_date,plan_name,churn_date
0,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,Unknown
1,1001,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,Unknown
2,1002,Carlee,Hoffman,36,"Las Vegas-Henderson-Paradise, NV MSA",2018-10-21,surf,Unknown
3,1003,Reynaldo,Jenkins,52,"Tulsa, OK MSA",2018-01-28,surf,Unknown
4,1004,Leonila,Thompson,40,"Seattle-Tacoma-Bellevue, WA MSA",2018-05-23,surf,Unknown
...,...,...,...,...,...,...,...,...
495,1495,Fidel,Sharpe,67,"New York-Newark-Jersey City, NY-NJ-PA MSA",2018-09-04,surf,Unknown
496,1496,Ariel,Shepherd,49,"New Orleans-Metairie, LA MSA",2018-02-20,surf,Unknown
497,1497,Donte,Barrera,49,"Los Angeles-Long Beach-Anaheim, CA MSA",2018-12-10,ultimate,Unknown
498,1498,Scot,Williamson,51,"New York-Newark-Jersey City, NY-NJ-PA MSA",2018-02-04,surf,Unknown


### Enriquecer los datos

Al tener una columna con la Fecha de Registro, podemos dividirla para extraer el mes y el año y agregarlos en nuevas columnas cada valor.
Esta información separada, nos podría ser útil para algun análisis más adelante.

In [10]:
# Separaremos los meses y los años de la fecha de registro
reg_month = df_users['reg_date'].dt.month
df_users.insert(6, 'reg_month', reg_month)   # agregamos una columna con el mes de registro

df_users    # imrpimimos el nuevo datafram

,user_id,first_name,last_name,user_age,user_city,reg_date,reg_month,plan_name,churn_date
0,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,12,ultimate,Unknown
1,1001,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,8,surf,Unknown
2,1002,Carlee,Hoffman,36,"Las Vegas-Henderson-Paradise, NV MSA",2018-10-21,10,surf,Unknown
3,1003,Reynaldo,Jenkins,52,"Tulsa, OK MSA",2018-01-28,1,surf,Unknown
4,1004,Leonila,Thompson,40,"Seattle-Tacoma-Bellevue, WA MSA",2018-05-23,5,surf,Unknown
...,...,...,...,...,...,...,...,...,...
495,1495,Fidel,Sharpe,67,"New York-Newark-Jersey City, NY-NJ-PA MSA",2018-09-04,9,surf,Unknown
496,1496,Ariel,Shepherd,49,"New Orleans-Metairie, LA MSA",2018-02-20,2,surf,Unknown
497,1497,Donte,Barrera,49,"Los Angeles-Long Beach-Anaheim, CA MSA",2018-12-10,12,ultimate,Unknown
498,1498,Scot,Williamson,51,"New York-Newark-Jersey City, NY-NJ-PA MSA",2018-02-04,2,surf,Unknown


## Llamadas

In [11]:
# Imprime la información general/resumida sobre el DataFrame de las llamadas
df_calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137735 entries, 0 to 137734
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         137735 non-null  object 
 1   user_id    137735 non-null  int64  
 2   call_date  137735 non-null  object 
 3   duration   137735 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.2+ MB


In [12]:
# Imprime una muestra de datos para las llamadas
df_calls.sample(5)

,id,user_id,call_date,duration
116477,1408_126,1408,2018-10-25,14.01
105252,1370_542,1370,2018-10-20,17.11
33209,1124_63,1124,2018-11-01,5.17
124093,1438_141,1438,2018-10-18,8.77
131849,1474_295,1474,2018-10-17,3.76


En el dataframe de llamadas, tenemos únicamente 4 columnas: 'id', 'user_id', 'call_date' y 'duration'.

Revisando los Dtypes podemos notar que la información de la columna 'call_date' está como un 'object' lo cual no es correcto, tendremos que corregir este formato. al igual que la duración de las llamadas está en minutos.

Se tiene un registro de 137,735 llamadas, sin datos ausentes o nulos.

### Corregir los datos

Vamos a cambiar el Dtype de la columna 'call_date' para convertirlo en formato de fecha.

También actualizaremos el nombre de las columnas.

In [13]:
df_calls['call_date'] = pd.to_datetime(df_calls['call_date'], infer_datetime_format=True)       # cambiamos el formato a fecha en la columna de 'call_date'
df_calls.columns = ['call_id', 'user_id', 'call_date', 'duration']
df_calls.info()    # revisamos la información general nuevamente
df_calls

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137735 entries, 0 to 137734
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   call_id    137735 non-null  object        
 1   user_id    137735 non-null  int64         
 2   call_date  137735 non-null  datetime64[ns]
 3   duration   137735 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 4.2+ MB


C:\Users\10147115\AppData\Local\Temp\ipykernel_13468\2140080087.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_calls['call_date'] = pd.to_datetime(df_calls['call_date'], infer_datetime_format=True)       # cambiamos el formato a fecha en la columna de 'call_date'


,call_id,user_id,call_date,duration
0,1000_93,1000,2018-12-27,8.52
1,1000_145,1000,2018-12-27,13.66
2,1000_247,1000,2018-12-27,14.48
3,1000_309,1000,2018-12-28,5.76
4,1000_380,1000,2018-12-30,4.22
...,...,...,...,...
137730,1499_199,1499,2018-11-21,8.72
137731,1499_200,1499,2018-10-20,10.89
137732,1499_201,1499,2018-09-21,8.12
137733,1499_202,1499,2018-10-10,0.37


### Enriquecer los datos

En este dataframe de llamadas podemos hacer varias cosas para enriquecer la información.

Primeramente vamos a separar el mes y el año de la fecha de cada llamada.

Seguido de separar la duración en minutos y segundos.

In [14]:
# Separaremos los meses y los años de la fecha de la llamada
call_month = df_calls['call_date'].dt.month
df_calls.insert(3, 'call_month', call_month)   # agregamos una columna con el mes de registro

# Vamos a convertir los minutos en segundos
df_calls['duration_sec'] = df_calls['duration'] * 60
df_calls.columns = ['call_id', 'user_id', 'call_date', 'call_month', 'duration_min', 'duration_sec']

df_calls    # imrpimimos el nuevo datafram

,call_id,user_id,call_date,call_month,duration_min,duration_sec
0,1000_93,1000,2018-12-27,12,8.52,511.2
1,1000_145,1000,2018-12-27,12,13.66,819.6
2,1000_247,1000,2018-12-27,12,14.48,868.8
3,1000_309,1000,2018-12-28,12,5.76,345.6
4,1000_380,1000,2018-12-30,12,4.22,253.2
...,...,...,...,...,...,...
137730,1499_199,1499,2018-11-21,11,8.72,523.2
137731,1499_200,1499,2018-10-20,10,10.89,653.4
137732,1499_201,1499,2018-09-21,9,8.12,487.2
137733,1499_202,1499,2018-10-10,10,0.37,22.2


## Mensajes

In [15]:
# Imprime la información general/resumida sobre el DataFrame de los mensajes
df_sms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76051 entries, 0 to 76050
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            76051 non-null  object
 1   user_id       76051 non-null  int64 
 2   message_date  76051 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


In [16]:
# Imprime una muestra de datos para los mensajes
df_sms.sample(5)

,id,user_id,message_date
6744,1057_68,1057,2018-10-25
68433,1442_5,1442,2018-12-18
73223,1470_851,1470,2018-11-06
5911,1055_29,1055,2018-10-11
29148,1178_220,1178,2018-10-29


En el dataframde de mensajes tipo "sms" tenemos 3 columnas que se llaman 'id', 'user_id' y 'message_date'.

Las columnas tienen Dtype object, int64 y objet, lo cual no es lo correcto al tener un dato de fecha como objeto, este debe ser corregido.

Se tiene un total de 76, 051 mensajes registrados, sin valores nulos.

### Corregir los datos

Vamos a cambiar el Dtype de la columna 'message_date' para convertirlo en formato de fecha.

Actualisaremos el nombre de las columnas para mejor identificación del id.

In [17]:
df_sms['message_date'] = pd.to_datetime(df_sms['message_date'], infer_datetime_format=True)       # cambiamos el formato a fecha en la columna de 'message_date'
df_sms.columns = ['sms_id', 'user_id', 'message_date']
df_sms. info()    # revisamos la información general nuevamente

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76051 entries, 0 to 76050
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   sms_id        76051 non-null  object        
 1   user_id       76051 non-null  int64         
 2   message_date  76051 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 1.7+ MB


C:\Users\10147115\AppData\Local\Temp\ipykernel_13468\402032001.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_sms['message_date'] = pd.to_datetime(df_sms['message_date'], infer_datetime_format=True)       # cambiamos el formato a fecha en la columna de 'message_date'


### Enriquecer los datos

En este dataframe de mensajes podemos separar el mes de la fecha de cada mensaje.

In [18]:
df_sms['message_month'] = df_sms['message_date'].dt.month   # agregamos una columna con el mes de registro
df_sms  # revisamos el nuevo dataframe

,sms_id,user_id,message_date,message_month
0,1000_125,1000,2018-12-27,12
1,1000_160,1000,2018-12-31,12
2,1000_223,1000,2018-12-31,12
3,1000_251,1000,2018-12-27,12
4,1000_255,1000,2018-12-26,12
...,...,...,...,...
76046,1497_526,1497,2018-12-24,12
76047,1497_536,1497,2018-12-24,12
76048,1497_547,1497,2018-12-31,12
76049,1497_558,1497,2018-12-24,12


## Internet

In [19]:
# Imprime la información general/resumida sobre el DataFrame de internet
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104825 entries, 0 to 104824
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            104825 non-null  object 
 1   user_id       104825 non-null  int64  
 2   session_date  104825 non-null  object 
 3   mb_used       104825 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.2+ MB


In [20]:
# Imprime una muestra de datos para el tráfico de internet
df_data.sample(5)

,id,user_id,session_date,mb_used
66787,1312_10,1312,2018-08-31,522.73
42124,1187_301,1187,2018-09-21,519.82
2803,1014_39,1014,2018-12-13,52.52
29463,1134_286,1134,2018-11-03,270.45
67748,1314_86,1314,2018-11-10,108.77


El dataframe de data sobre internet, se pueden notar 4 columnas principales llamadas 'id', 'user_id', 'session_date' y 'mb_user'.

Las columnas tienen un Dtype de 'object', 'int64', 'object' y 'float64' respectivamente para cada una, al igual que en los otros dataframes, es conveniente corregir el Dtype para la informacion que contenga fechas.

El total de registros de consultas en internet, es de 104,825 líneas de las cuales no hay valores ausentes.

### Corregir los datos

Vamos a cambiar el Dtype de la columna 'session_date' para convertirlo en formato de fecha.

Actualisaremos el nombre de las columnas para mejor identificación del id.

In [21]:
df_data['session_date'] = pd.to_datetime(df_data['session_date'], infer_datetime_format=True)       # cambiamos el formato a fecha en la columna de 'message_date'
df_data.columns = ['data_id', 'user_id', 'session_date', 'mb_used']
df_data.info()    # revisamos la información general nuevamente

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104825 entries, 0 to 104824
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   data_id       104825 non-null  object        
 1   user_id       104825 non-null  int64         
 2   session_date  104825 non-null  datetime64[ns]
 3   mb_used       104825 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 3.2+ MB


C:\Users\10147115\AppData\Local\Temp\ipykernel_13468\1893009059.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_data['session_date'] = pd.to_datetime(df_data['session_date'], infer_datetime_format=True)       # cambiamos el formato a fecha en la columna de 'message_date'


### Enriquecer los datos

Vamos a separar el mes y el año de la fecha de cada consulta en internet.

In [22]:
# Separaremos los meses y los años de la fecha de la llamada
data_month = df_data['session_date'].dt.month
df_data.insert(3, 'session_month', data_month)   # agregamos una columna con el mes de la consulta

df_data.sample(5)      # imprimimos 5 líneas aleatorias del dataframe para ver cómo quedó

,data_id,user_id,session_date,session_month,mb_used
95353,1445_1,1445,2018-05-29,5,0.00
98959,1467_96,1467,2018-07-23,7,170.15
4887,1028_458,1028,2018-12-16,12,862.85
58194,1262_46,1262,2018-12-27,12,622.94
37276,1167_199,1167,2018-09-14,9,88.97


## Estudiar las condiciones de las tarifas

[Es sumamente importante entender cómo funcionan las tarifas, cómo se les cobra a los usuarios en función de su plan de suscripción. Así que te sugerimos imprimir la información de la tarifa para ver una vez más sus condiciones.]

# Imprime las condiciones de la tarifa y asegúrate de que te quedan claras
Descripción de las tarifas:

Nota: Megaline redondea los segundos a minutos y los megabytes a gigabytes. Para las llamadas, cada llamada individual se redondea: incluso si la llamada duró solo un segundo, se contará como un minuto. Para el tráfico web, las sesiones web individuales no se redondean. En vez de esto, el total del mes se redondea hacia arriba. Si alguien usa 1025 megabytes este mes, se le cobrarán 2 gigabytes.

A continuación, puedes ver una descripción de las tarifas:

Surf

1.	Pago mensual: $20 usd.
2.	500 minutos al mes, 50 SMS y 15 GB de datos.
3.	Si se exceden los límites del paquete:

    o	1 minuto: 3 centavos.

    o	1 SMS: 3 centavos.

    o	1 GB de datos: $10 usd.

Ultimate

1.	Pago mensual: $70 usd.
2.	3000 minutos al mes, 1000 SMS y 30 GB de datos.
3.	Si se exceden los límites del paquete:

    o	1 minuto: 1 centavo.

    o	1 SMS: 1 centavo.
    
    o	1 GB de datos: $7 usd.

## Agregar datos por usuario

[Ahora que los datos están limpios, agrega los datos por usuario y por periodo para que solo haya un registro por usuario y por periodo. Esto facilitará mucho el análisis posterior.]

In [23]:
# Calcula el número de llamadas hechas por cada usuario al mes. Guarda el resultado.
month_user_calls = df_calls.groupby(['user_id', 'call_month'])['call_id'].count()
month_user_calls.info()
month_user_calls

<class 'pandas.core.series.Series'>
MultiIndex: 2258 entries, (np.int64(1000), np.int32(12)) to (np.int64(1499), np.int32(12))
Series name: call_id
Non-Null Count  Dtype
--------------  -----
2258 non-null   int64
dtypes: int64(1)
memory usage: 28.2 KB


user_id  call_month
1000     12            16
1001     8             27
         9             49
         10            65
         11            64
                       ..
1498     12            39
1499     9             41
         10            53
         11            45
         12            65
Name: call_id, Length: 2258, dtype: int64

In [24]:
# Calcula la cantidad de minutos usados por cada usuario al mes. Guarda el resultado.
month_user_minutes = df_calls.groupby(['user_id', 'call_month'])['duration_min'].sum()
month_user_minutes.info()
month_user_minutes

<class 'pandas.core.series.Series'>
MultiIndex: 2258 entries, (np.int64(1000), np.int32(12)) to (np.int64(1499), np.int32(12))
Series name: duration_min
Non-Null Count  Dtype  
--------------  -----  
2258 non-null   float64
dtypes: float64(1)
memory usage: 28.2 KB


user_id  call_month
1000     12            116.83
1001     8             171.14
         9             297.69
         10            374.11
         11            404.59
                        ...  
1498     12            324.77
1499     9             330.37
         10            363.28
         11            288.56
         12            468.10
Name: duration_min, Length: 2258, dtype: float64

In [25]:
# Calcula el número de mensajes enviados por cada usuario al mes. Guarda el resultado.
month_user_sms = df_sms.groupby(['user_id', 'message_month'])['sms_id'].count()
month_user_sms.info()
month_user_sms

<class 'pandas.core.series.Series'>
MultiIndex: 1806 entries, (np.int64(1000), np.int32(12)) to (np.int64(1497), np.int32(12))
Series name: sms_id
Non-Null Count  Dtype
--------------  -----
1806 non-null   int64
dtypes: int64(1)
memory usage: 22.7 KB


user_id  message_month
1000     12               11
1001     8                30
         9                44
         10               53
         11               36
                          ..
1496     9                21
         10               18
         11               13
         12               11
1497     12               50
Name: sms_id, Length: 1806, dtype: int64

In [26]:
# Calcula el volumen del tráfico de Internet usado por cada usuario al mes. Guarda el resultado.
month_user_data = df_data.groupby(['user_id', 'session_month'])['mb_used'].sum()
month_user_data.info()
month_user_data

<class 'pandas.core.series.Series'>
MultiIndex: 2277 entries, (np.int64(1000), np.int32(12)) to (np.int64(1499), np.int32(12))
Series name: mb_used
Non-Null Count  Dtype  
--------------  -----  
2277 non-null   float64
dtypes: float64(1)
memory usage: 28.4 KB


user_id  session_month
1000     12                1901.47
1001     8                 6919.15
         9                13314.82
         10               22330.49
         11               18504.30
                            ...   
1498     12               23137.69
1499     9                12984.76
         10               19492.43
         11               16813.83
         12               22059.21
Name: mb_used, Length: 2277, dtype: float64

[Junta los datos agregados en un DataFrame para que haya un registro que represente lo que consumió un usuario único en un mes determinado.]

In [27]:
# Fusiona los datos de llamadas, minutos, mensajes e Internet con base en user_id y month
tripleplay = pd.concat([month_user_calls, month_user_minutes, month_user_sms, month_user_data], axis=1).reset_index()
tripleplay.columns = ['user_id', 'month', 'calls', 'minutes', 'sms', 'mb']    # Actualizamos los nombres de las columnas para mejor entendimiento

# Modificamos los Dtypes para corregirlos
tripleplay['user_id'] = tripleplay['user_id'].astype('object')
tripleplay['month'] = tripleplay['month'].astype('int64')

tripleplay.info()   # Imprimimos información general y el nuevo dataframe
tripleplay

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2293 entries, 0 to 2292
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   user_id  2293 non-null   object 
 1   month    2293 non-null   int64  
 2   calls    2258 non-null   float64
 3   minutes  2258 non-null   float64
 4   sms      1806 non-null   float64
 5   mb       2277 non-null   float64
dtypes: float64(4), int64(1), object(1)
memory usage: 107.6+ KB


,user_id,month,calls,minutes,sms,mb
0,1000,12,16.0,116.83,11.0,1901.47
1,1001,8,27.0,171.14,30.0,6919.15
2,1001,9,49.0,297.69,44.0,13314.82
3,1001,10,65.0,374.11,53.0,22330.49
4,1001,11,64.0,404.59,36.0,18504.30
...,...,...,...,...,...,...
2288,1349,12,NaN,NaN,61.0,13039.91
2289,1361,5,NaN,NaN,2.0,1519.69
2290,1482,10,NaN,NaN,2.0,NaN
2291,1108,12,NaN,NaN,NaN,233.17


In [28]:
# Añade la información de la tarifa
tripleplay_plan = tripleplay.merge(df_users, on='user_id')    # combinamos los dataframes de 'all_services' con 'df_users' por el número de id
tripleplay_plan.drop(['first_name', 'last_name', 'user_age', 'user_city', 'reg_date', 'reg_month', 'churn_date'], axis='columns', inplace=True)   # Eliminamos las columnas inecesarias
tripleplay_fee = tripleplay_plan.merge(df_plans, on='plan_name')    # combinamos el nuevo dataframe con 'df_plans' para agregar la información de las tarifas
tripleplay_fee.head()  # imprimimos una muestra de 10 lineas para revisar

,user_id,month,calls,minutes,sms,mb,plan_name,messages_included,mb_per_month_included,gb_included,minutes_included,usd_monthly_pay,usd_per_gb,usd_per_message,usd_per_minute
0,1000,12,16.0,116.83,11.0,1901.47,ultimate,1000,30720,30,3000,70.0,7.0,0.01,0.01
1,1001,8,27.0,171.14,30.0,6919.15,surf,50,15360,15,500,20.0,10.0,0.03,0.03
2,1001,9,49.0,297.69,44.0,13314.82,surf,50,15360,15,500,20.0,10.0,0.03,0.03
3,1001,10,65.0,374.11,53.0,22330.49,surf,50,15360,15,500,20.0,10.0,0.03,0.03
4,1001,11,64.0,404.59,36.0,18504.30,surf,50,15360,15,500,20.0,10.0,0.03,0.03


In [29]:
# Filtramos los usuarios gastalones de minutos que tienen plan ultimate
ultimate_users_extra = tripleplay_fee[(tripleplay_fee['plan_name'] == 'ultimate') & (tripleplay_fee['minutes'] > 3000)]

# Filtramos los usuarios gastalones del minutos que tienen plan surf
surf_users_extra = tripleplay_fee[(tripleplay_fee['plan_name'] == 'surf') & (tripleplay_fee['minutes'] > 500)]
surf_users_extra

,user_id,month,calls,minutes,sms,mb,plan_name,messages_included,mb_per_month_included,gb_included,minutes_included,usd_monthly_pay,usd_per_gb,usd_per_message,usd_per_minute
9,1003,12,149.0,1041.00,50.0,27044.14,surf,50,15360,15,500,20.0,10.0,0.03,0.03
23,1007,10,80.0,610.65,59.0,37885.63,surf,50,15360,15,500,20.0,10.0,0.03,0.03
25,1007,12,87.0,579.43,50.0,29069.28,surf,50,15360,15,500,20.0,10.0,0.03,0.03
30,1009,6,110.0,775.52,NaN,18845.90,surf,50,15360,15,500,20.0,10.0,0.03,0.03
31,1009,7,124.0,826.74,NaN,30421.65,surf,50,15360,15,500,20.0,10.0,0.03,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2197,1488,11,109.0,751.66,NaN,19159.52,surf,50,15360,15,500,20.0,10.0,0.03,0.03
2198,1488,12,96.0,678.14,NaN,19805.73,surf,50,15360,15,500,20.0,10.0,0.03,0.03
2215,1491,10,75.0,551.89,51.0,8480.09,surf,50,15360,15,500,20.0,10.0,0.03,0.03
2230,1494,10,78.0,585.82,38.0,14498.50,surf,50,15360,15,500,20.0,10.0,0.03,0.03


[Calcula los ingresos mensuales por usuario (resta el límite del paquete gratuito del número total de llamadas, mensajes de texto y datos; multiplica el resultado por el valor del plan de llamadas; añade la tarifa mensual en función del plan de llamadas). Nota: Dadas las condiciones del plan, ¡esto podría no ser tan trivial como un par de líneas! Así que no pasa nada si dedicas algo de tiempo a ello.]

In [30]:
# Calcula el ingreso mensual para cada usuario

# Variables y formula para llamadas (minutos) extra
plan_charge_min = tripleplay_fee['usd_monthly_pay']
usd_per_minute = tripleplay_fee['usd_per_minute']

extra_min = (tripleplay_fee['minutes']) - (tripleplay_fee['minutes_included'])
extra_charge = extra_min * usd_per_minute

# Fórmula para cargos extra
charge_user_calls = (plan_charge_min) + (extra_min * extra_charge)

In [ ]:
if tripleplay_fee['plan_name'] == 'surf':
    charge_user_calls = (plan_charge_min) + (extra_min * extra_charge)
else:
    ultimate_users_extra

## Estudia el comportamiento de usuario

[Calcula algunas estadísticas descriptivas para los datos agregados y fusionados que nos sean útiles y que muestren un panorama general captado por los datos. Dibuja gráficos útiles para facilitar la comprensión. Dado que la tarea principal es comparar las tarifas y decidir cuál es más rentable, las estadísticas y gráficas deben calcularse por tarifa.]

[En los comentarios hallarás pistas relevantes para las llamadas, pero no las hay para los mensajes e Internet. Sin embargo, el principio del estudio estadístico que se aplica para ellos es el mismo que para las llamadas.]

### Llamadas

In [ ]:
# Compara la duración promedio de llamadas por cada plan y por cada mes. Traza un gráfico de barras para visualizarla.



In [ ]:
# Compara el número de minutos mensuales que necesitan los usuarios de cada plan. Traza un histograma.



[Calcula la media y la variable de la duración de las llamadas para averiguar si los usuarios de los distintos planes se comportan de forma diferente al realizar sus llamadas.]

In [ ]:
# Calcula la media y la varianza de la duración mensual de llamadas.



In [ ]:
# Traza un diagrama de caja para visualizar la distribución de la duración mensual de llamadas



[Elabora las conclusiones sobre el comportamiento de los usuarios con respecto a las llamadas. ¿Su comportamiento varía en función del plan?]

### Mensajes

In [ ]:
# Comprara el número de mensajes que tienden a enviar cada mes los usuarios de cada plan



In [ ]:
# Compara la cantidad de tráfico de Internet consumido por usuarios por plan



[Elabora las conclusiones sobre el comportamiento de los usuarios con respecto a los mensajes. ¿Su comportamiento varía en función del plan?]

### Internet

[Elabora las conclusiones sobre cómo los usuarios tienden a consumir el tráfico de Internet. ¿Su comportamiento varía en función del plan?]

## Ingreso

[Del mismo modo que has estudiado el comportamiento de los usuarios, describe estadísticamente los ingresos de los planes.]

[Elabora las conclusiones sobre cómo difiere el ingreso entre los planes.]

## Prueba las hipótesis estadísticas

[Prueba la hipótesis de que son diferentes los ingresos promedio procedentes de los usuarios de los planes de llamada Ultimate y Surf.]

[Elabora las hipótesis nula y alternativa, escoge la prueba estadística, determina el valor alfa.]

In [ ]:
# Prueba las hipótesis



[Prueba la hipótesis de que el ingreso promedio de los usuarios del área NY-NJ es diferente al de los usuarios de otras regiones.]

[Elabora las hipótesis nula y alternativa, escoge la prueba estadística, determina el valor alfa.]

In [ ]:
# Prueba las hipótesis



## Conclusión general

[En esta sección final, enumera tus conclusiones importantes. Asegúrate de que estas abarquen todas las decisiones (suposiciones) importantes que adoptaste y que determinaron la forma elegida para procesar y analizar los datos.]